<a href="https://colab.research.google.com/github/jonxu88/CodeFromColab/blob/main/TensorFlowClassification2XceptionUsingAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Classification of CIFAR10, by implementing Xception, directly from the Keras API
#No weights preloaded

import tensorflow as tf
import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential as Sequential
import numpy as np
import matplotlib.pyplot as plt

cifar10 = tf.keras.datasets.cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

#Labels are just from 0 to 9, so let's store the actual class names
class_names =['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


170500096/170498071 [==============================] - 4s 0us/step


In [2]:
#Inspect images/dimensions etc

# Resize pictures from 32x32 to 299x299 to be accepted as input by Xception


# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(train_images[i], cmap=plt.cm.binary)
#     plt.xlabel(class_names[int(train_labels[i])])
# plt.show()

# plt.figure()
# plt.imshow(a)
# plt.colorbar()
# plt.grid(False)
# plt.show()

# plt.figure(figsize=(10,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(train_images[i], cmap=plt.cm.binary)
#     plt.xlabel(class_names[int(train_labels[i])])
# plt.show()




In [3]:
#Currently, train_images consists of 50000 different 32 by 32 by 3 matrices, with entries an integer between 0 and 255
#The type of these integers is 'numpy.uint8'. It is now converted into a float32.

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images = (train_images - 127.5)/127.5
test_images = (test_images -127.5)/127.5

print(train_images[0])


[[[-0.5372549  -0.5137255  -0.5058824 ]
  [-0.6627451  -0.6392157  -0.64705884]
  [-0.60784316 -0.62352943 -0.6627451 ]
  ...
  [ 0.23921569  0.03529412 -0.15294118]
  [ 0.19215687 -0.01960784 -0.2       ]
  [ 0.16078432 -0.02745098 -0.19215687]]

 [[-0.8745098  -0.84313726 -0.84313726]
  [-1.         -1.         -1.        ]
  [-0.85882354 -0.9372549  -1.        ]
  ...
  [-0.03529412 -0.30980393 -0.5686275 ]
  [-0.06666667 -0.34901962 -0.60784316]
  [-0.04313726 -0.31764707 -0.5529412 ]]

 [[-0.8039216  -0.8117647  -0.8352941 ]
  [-0.8745098  -0.94509804 -1.        ]
  [-0.6156863  -0.7882353  -0.9372549 ]
  ...
  [-0.07450981 -0.34117648 -0.60784316]
  [-0.05882353 -0.34117648 -0.60784316]
  [-0.14509805 -0.42745098 -0.67058825]]

 ...

 [[ 0.6313726   0.33333334 -0.24705882]
  [ 0.5764706   0.2        -0.73333335]
  [ 0.5529412   0.2627451  -0.79607844]
  ...
  [ 0.25490198  0.04313726 -0.4509804 ]
  [-0.56078434 -0.75686276 -0.94509804]
  [-0.58431375 -0.73333335 -0.84313726]]

 [

In [4]:
#Note: Xception doesn't seem to like any input dimension less than 71, for example, in our case, 32.
#There is extra stuff here to add a preprocessing layer to the beginning of the xception model
#which converts the 32 x 32 images to 96 x 96 images.


inputs = keras.Input(shape=(32,32,3))
preprocessing_layer = tf.keras.layers.experimental.preprocessing.Resizing(96,96)
x = preprocessing_layer(inputs)
#Import Xception from the Keras API
xception_model_without_preprocessing = keras.applications.Xception(weights=None, input_shape=(96,96,3), classes=10)

outputs = xception_model_without_preprocessing(x)
xception_model = keras.Model(inputs,outputs)


xception_model.summary()





Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
resizing (Resizing)          (None, 96, 96, 3)         0         
_________________________________________________________________
xception (Functional)        (None, 10)                20881970  
Total params: 20,881,970
Trainable params: 20,827,442
Non-trainable params: 54,528
_________________________________________________________________


In [5]:
#Compile the model
model = xception_model

model.compile(optimizer=keras.optimizers.Adam(lr=0.01), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=15)

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

Epoch 1/15
1563/1563 [==============================] - 134s 79ms/step - loss: 2.4252 - accuracy: 0.2228
Epoch 2/15
1563/1563 [==============================] - 122s 78ms/step - loss: 1.4745 - accuracy: 0.4606
Epoch 3/15
1563/1563 [==============================] - 122s 78ms/step - loss: 1.1164 - accuracy: 0.5995
Epoch 4/15
1563/1563 [==============================] - 122s 78ms/step - loss: 0.8316 - accuracy: 0.7099
Epoch 5/15
1563/1563 [==============================] - 122s 78ms/step - loss: 0.6410 - accuracy: 0.7776
Epoch 6/15
1563/1563 [==============================] - 122s 78ms/step - loss: 0.5202 - accuracy: 0.8206
Epoch 7/15
1563/1563 [==============================] - 121s 78ms/step - loss: 0.4133 - accuracy: 0.8536
Epoch 8/15
1563/1563 [==============================] - 121s 78ms/step - loss: 0.3417 - accuracy: 0.8797
Epoch 9/15
1563/1563 [==============================] - 121s 78ms/step - loss: 0.2721 - accuracy: 0.9045
Epoch 10/15
1563/1563 [==============================] 